<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Langgraph_Agentic_RAG_Cyber_AI_Copilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Cyber AI Copilot for Security and Intelligence Domain**

In [1]:
!pip install --upgrade --quiet sentence-transformers langchain langchain-groq langchain-pinecone langchain-core asknews langgraph
%pip install --quiet -U "langchain-community>=0.2.16"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24

In [2]:
import os
from typing import List, Dict, Any, Optional, TypedDict
from pydantic import BaseModel
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langgraph.graph import StateGraph, END
from langchain_community.tools.asknews import AskNewsSearch
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.tools import JinaSearch
from dotenv import load_dotenv
import re
from datetime import datetime, timedelta

# Load environment variables
load_dotenv()

# API Keys (hidden for security purposes)
GROQ_API_KEY = "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"
PINECONE_API_KEY = "8e15b925-3b96-497d-b20a-08d308782b83"
PINECONE_ENVIRONMENT = "us-east-1"
ASKNEWS_CLIENT_ID = "a0de4609-b760-4c83-9609-5c04d7743b84"
ASKNEWS_CLIENT_SECRET = "D5Mlhkztk4TcW24diUgcW0FA2w"
SERPER_API_KEY = "d8e815ef6caa94dbef7b977a0ea7d505b43a5a06"

# Set environment variables for AskNews
os.environ["ASKNEWS_CLIENT_ID"] = ASKNEWS_CLIENT_ID
os.environ["ASKNEWS_CLIENT_SECRET"] = ASKNEWS_CLIENT_SECRET
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [ ]:
# Initialize the LLM and embeddings
llm = ChatGroq(temperature=0, model="llama3-8b-8192", api_key=GROQ_API_KEY)
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

# Initialize Pinecone and vector store
pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
pinecone_index = pc.Index("new-cyber-search")
vector_store = PineconeVectorStore(index=pinecone_index, embedding=embeddings)

# Initialize AskNews tool
asknews_tool = AskNewsSearch(max_results=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Define the state
class AgentState(TypedDict):
    messages: List[Dict[str, str]]
    memory: Optional[Dict[str, Any]]

def vector_search(query: str) -> str:
    results = vector_store.similarity_search(query, k=3)
    return "\n".join([f"Source {i+1}: {doc.page_content}" for i, doc in enumerate(results)])

# Define the nodes
def search_asknews(state: AgentState) -> AgentState:
    query = state["messages"][-1]["content"]
    result = asknews_tool.invoke({"query": query})
    state["messages"].append({"role": "tool", "content": f"AskNews Search Result: {result}"})
    return state

def search_vector(state: AgentState) -> AgentState:
    query = state["messages"][-1]["content"]
    result = vector_search(query)
    state["messages"].append({"role": "tool", "content": f"Vector Search Result: {result}"})
    return state

In [ ]:
def generate_response(state: AgentState) -> AgentState:
    memory = state.get("memory", {})
    chat_history = memory.get("chat_history", "")

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an advanced AI copilot specializing in cybersecurity and intelligence. Your role is to provide accurate, up-to-date information and insights on cyber threats, vulnerabilities, and intelligence matters. Use the following guidelines:

1. Analyze the user's query and the provided search results thoroughly.
2. Prioritize the most relevant and recent information from both AskNews and vector search results.
3. Provide a comprehensive response that addresses the user's query, incorporating insights from both search sources.
4. Structure your response clearly with headings and bullet points where appropriate.
5. If there are conflicting pieces of information, acknowledge them and provide a balanced view.
6. Always consider the ethical implications of the information you're providing, especially in sensitive cybersecurity matters.
7. If asked about specific attack techniques or vulnerabilities, provide information for defensive purposes only.
8. Recommend reliable sources for further reading when appropriate.
9. If you're unsure about any information, clearly state that and suggest ways to verify or obtain more accurate data.
10. Do not hallucinate or make up information. If you don't have enough information to answer a question, say so.

Remember, you're assisting in a professional capacity in the cybersecurity and intelligence domain. Maintain a tone that is informative, analytical, and security-conscious.

Previous conversation:
{chat_history}

New human question: {input}
AskNews Search Result: {asknews_result}
Vector Search Result: {vector_result}

Response:"""),
    ])

    chain = prompt | llm

    response = chain.invoke({
        "input": state["messages"][-1]["content"],
        "asknews_result": next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "AskNews" in m["content"]), "No AskNews results."),
        "vector_result": next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "Vector Search" in m["content"]), "No vector search results."),
        "chat_history": chat_history
    })

    state["messages"].append({"role": "assistant", "content": response.content})
    state["memory"] = {"chat_history": chat_history + f"\nHuman: {state['messages'][-2]['content']}\nAI: {response.content}"}
    return state

# Define the graph
workflow = StateGraph(AgentState)

# Define the nodes
workflow.add_node("search_asknews", search_asknews)
workflow.add_node("search_vector", search_vector)
workflow.add_node("generate_response", generate_response)

# Define the edges
workflow.add_edge("search_asknews", "search_vector")
workflow.add_edge("search_vector", "generate_response")
workflow.add_edge("generate_response", END)

# Set the entry point
workflow.set_entry_point("search_asknews")

# Compile the graph
graph = workflow.compile()

# Function to run the agent
def run_agent(query: str, memory: Optional[Dict[str, Any]] = None) -> AgentState:
    state = AgentState(messages=[{"role": "human", "content": query}], memory=memory or {})
    result = graph.invoke(state)
    return result

In [ ]:
# Example usage
if __name__ == "__main__":
    query = "List Down all attacks from Blackbasta Ransomeware last One Year?"
    result = run_agent(query)
    for message in result["messages"]:
        if message["role"] == "assistant":
            print("AI Copilot:", message["content"])

AI Copilot: **Ransomware Trends and Insights**

The recent surge in ransomware attacks has been a significant concern for organizations across various industries. According to ReliaQuest, there has been a 35% increase in ransomware victims from the professional, scientific, and technical services (PSTS) sector, which includes software companies. This trend is consistent with the findings of Secureworks' 2024 State of the Threat Report, which revealed a 30% year-over-year rise in active ransomware groups.

**Key Takeaways:**

* Ransomware attacks are becoming more frequent and sophisticated, with the increasing use of AI-powered attacks and Ransomware-as-a-Service models.
* The manufacturing industry has been the most targeted, accounting for more than double any other industry.
* The healthcare sector has also been severely affected, with 67% of organizations reporting ransomware attacks in 2024.
* Zero-trust architecture and AI-powered zero-trust platforms are becoming essential for o